# Project

In [1]:
import pandas as pd
import datetime as dt
from sklearn.model_selection import GroupShuffleSplit

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

import spacy
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_postgres import PGVector
from langchain_community.vectorstores.pgvector import DistanceStrategy

from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, roc_auc_score

import json
import requests

nlp = spacy.load("en_core_web_sm", exclude=["ner"])

## Data

In [7]:
admissions = pd.read_csv("archive/MIMIC -III (10000 patients)/ADMISSIONS/ADMISSIONS_sorted.csv")
patients = pd.read_csv("archive/MIMIC -III (10000 patients)/PATIENTS/PATIENTS_sorted.csv")
diagnoses_icd = pd.read_csv("archive/MIMIC -III (10000 patients)/DIAGNOSES_ICD/DIAGNOSES_ICD_sorted.csv")
prescriptions = pd.read_csv("archive/MIMIC -III (10000 patients)/PRESCRIPTIONS/PRESCRIPTIONS_sorted.csv")
noteevents = pd.read_csv("archive/MIMIC -III (10000 patients)/NOTEEVENTS/NOTEEVENTS_sorted.csv")

C:\Users\krist\AppData\Local\Temp\ipykernel_47344\1491054095.py:4: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  prescriptions = pd.read_csv("archive/MIMIC -III (10000 patients)/PRESCRIPTIONS/PRESCRIPTIONS_sorted.csv")


## Label Extraction

Prescriptions table only has specific drug, not whether drug is opioid/non-opioid analgesic as in Garza et al.

In [8]:
opioid_keywords = [
    "buprenorphine", "butorphanol", "codeine", "fentanyl", "hydrocodone",
    "hydromorphone", "levorphanol", "meperidine", "methadone", "morphine",
    "nalbuphine", "oliceridine", "oxycodone", "oxymorphone", "remifentanil",
    "sufentanil", "tapentadol", "tramadol"
]

nonopioid_keywords = [
    "acetaminophen", "aspirin", "clonidine", "ibuprofen", "magnesium salicylate",
    "naproxen", "ziconotide"
]

In [9]:
prescriptions["is_opioid"] = prescriptions["DRUG"].str.contains("|".join(opioid_keywords), regex=True, case=False)
prescriptions["is_nonopioid"] = ~prescriptions["DRUG"].str.contains("|".join(opioid_keywords), regex=True, case=False) & prescriptions["DRUG"].str.contains("|".join(nonopioid_keywords), regex=True, case=False)
labels = prescriptions.groupby(["HADM_ID"])[["is_opioid", "is_nonopioid"]].any()

## Feature Extraction

### Age

In [10]:
merged_structured = admissions.merge(patients[["SUBJECT_ID", "DOB", "GENDER"]], how="left", on="SUBJECT_ID")
merged_structured["age"] = pd.Series((pd.to_datetime(merged_structured["ADMITTIME"]).dt.to_pydatetime() - pd.to_datetime(merged_structured["DOB"]).dt.to_pydatetime()) // dt.timedelta(days=365))

C:\Users\krist\AppData\Local\Temp\ipykernel_47344\2444041108.py:2: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  merged_structured["age"] = pd.Series((pd.to_datetime(merged_structured["ADMITTIME"]).dt.to_pydatetime() - pd.to_datetime(merged_structured["DOB"]).dt.to_pydatetime()) // dt.timedelta(days=365))
C:\Users\krist\AppData\Local\Temp\ipykernel_47344\2444041108.py:2: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  merged_structured["age"] = pd.Series((pd.to_datetime(merged_structured["ADMITTIME"]).dt.to_pydatetime() - pd.to_datetime(merged_structured["DOB"]).dt.to_pydatetime()) // dt.timedelta(day

### Race

Some ethnicity entries were specific (e.g. ASIAN - FILIPINO), made them more general.

In [11]:
merged_structured["ETHNICITY_general"] = merged_structured["ETHNICITY"].str.split(" - ").str[0]

### Housing status

ICD9_CODE V600=Homeless

In [12]:
diagnoses_icd["housed"] = (diagnoses_icd["ICD9_CODE"] != "V600")
housing_status = diagnoses_icd.groupby(["HADM_ID"])["housed"].all()
merged_structured = merged_structured.merge(housing_status, how="left", left_on="HADM_ID", right_index=True)

### Insurance status

Self Pay=Uninsured

In [13]:
merged_structured["INSURANCE_general"] = (merged_structured["INSURANCE"] != "Self Pay")

### Recidivism

In [14]:
recidivism = merged_structured.groupby("SUBJECT_ID")["HADM_ID"].count()
recidivism.name = "recidivism"
merged_structured = merged_structured.merge(recidivism, how="left", on="SUBJECT_ID")

## Data Split

In [15]:
merged_structured = merged_structured.merge(labels, how="left", left_on="HADM_ID", right_index=True)
merged_structured["is_opioid"] = merged_structured["is_opioid"].fillna(False)
merged_structured["is_nonopioid"] = merged_structured["is_nonopioid"].fillna(False)

C:\Users\krist\AppData\Local\Temp\ipykernel_47344\3033905579.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_structured["is_opioid"] = merged_structured["is_opioid"].fillna(False)
C:\Users\krist\AppData\Local\Temp\ipykernel_47344\3033905579.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_structured["is_nonopioid"] = merged_structured["is_nonopioid"].fillna(False)


In [16]:
#Train-(Test+Val) split
splitter = GroupShuffleSplit(test_size=0.3, n_splits=2, random_state=351)
split = splitter.split(merged_structured, groups=merged_structured["SUBJECT_ID"])
train_inds, test_val_inds = next(split)

train = merged_structured.iloc[train_inds]
test_val = merged_structured.iloc[test_val_inds]

#Test-Val split
splitter = GroupShuffleSplit(test_size=0.5, n_splits=2, random_state=351)
split = splitter.split(test_val, groups=test_val["SUBJECT_ID"])
test_inds, val_inds = next(split)

test = test_val.iloc[test_inds]
val = test_val.iloc[val_inds]

## Data Ingestion and Preprocessing

### Structured Data Normalization

#### Binning

In [17]:
def age_group(age):
    if age < 18:
        return 0
    elif age < 30:
        return 1
    elif age < 50:
        return 2
    elif age < 65:
        return 3
    else:
        return 4
    
def recidivism_group(recidivism):
    if recidivism < 10:
        return 0
    else:
        return 1

In [18]:
train["age_group"] = train["age"].map(age_group)
test["age_group"] = test["age"].map(age_group)
val["age_group"] = val["age"].map(age_group)

train["recidivism_group"] = train["recidivism"].map(recidivism_group)
test["recidivism_group"] = test["recidivism"].map(recidivism_group)
val["recidivism_group"] = val["recidivism"].map(recidivism_group)

C:\Users\krist\AppData\Local\Temp\ipykernel_47344\2441426422.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["age_group"] = train["age"].map(age_group)
C:\Users\krist\AppData\Local\Temp\ipykernel_47344\2441426422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["age_group"] = test["age"].map(age_group)
C:\Users\krist\AppData\Local\Temp\ipykernel_47344\2441426422.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

#### One-hot encoding

Preprocessing for baseline models:

In [14]:
preprocess_baseline = ColumnTransformer([
    ('cat', OneHotEncoder(drop="if_binary", handle_unknown="ignore", sparse_output=False), ["GENDER", "ETHNICITY_general", "INSURANCE_general", "housed"]),
    ('num', "passthrough", ["age_group", "recidivism_group"])],
    remainder='drop', verbose_feature_names_out=False)


Preprocessing for RAG (indicates metadata fields):

In [19]:
metadata_fields = ["SUBJECT_ID", "HADM_ID", "ADMITTIME", "DISCHTIME", "is_opioid", "is_nonopioid"]

preprocess_rag = ColumnTransformer([
    ('cat', OneHotEncoder(drop="if_binary", handle_unknown="ignore", sparse_output=False), ["GENDER", "ETHNICITY_general", "INSURANCE_general", "housed"]),
    ('num', "passthrough", ["age_group", "recidivism_group"]+metadata_fields)],
    remainder='drop', verbose_feature_names_out=False)

### Data Chunking & Embedding Preparation

In [20]:
preprocess_rag.fit(train)
train_preprocessed = pd.DataFrame(preprocess_rag.transform(train), columns=preprocess_rag.get_feature_names_out())

In [21]:
def create_entry(row):
    structured_content = "\n".join([f"{field}:{int(row[field])}" for field in row.index if field not in metadata_fields])
    
    unstructured_data = noteevents.loc[noteevents["HADM_ID"] == row["HADM_ID"]].to_dict()
    unstructured_content = "\n".join([f"<{unstructured_data["CATEGORY"][index]} {unstructured_data["DESCRIPTION"][index]}>\n{unstructured_data["TEXT"][index]}" for index in unstructured_data["TEXT"]])
    
    page_content = f"{structured_content}\n\n{unstructured_content}"
    metadata = {field: row[field] for field in row.index if field in metadata_fields}

    return page_content, metadata

In [2]:
embeddings = HuggingFaceEmbeddings(model_name="medicalai/ClinicalBERT")

user = "postgres"
password = "postgres"
host = "localhost"
port = "5432"
dbname = "cds_rag"

vector_store = PGVector(
    embeddings=embeddings,
    collection_name="train",
    connection=f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}",
    distance_strategy=DistanceStrategy.COSINE
)

text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20, length_function=lambda text: len(list(nlp(text).sents)))

No sentence-transformers model found with name medicalai/ClinicalBERT. Creating a new one with mean pooling.


In [20]:
for _, row in train_preprocessed.iterrows():
    if len(vector_store.similarity_search("", k=1, filter={"HADM_ID": {"$eq": row["HADM_ID"]}})) > 0:
        print(f"HADM_ID {row['HADM_ID']} already in database")
        continue

    page_content, metadata = create_entry(row)
    splitted = text_splitter.create_documents(texts=[page_content], metadatas=[metadata])
    vector_store.add_documents(splitted)
    print(f"HADM_ID {row['HADM_ID']}: {len(splitted)} chunks")

HADM_ID 145834: 6 chunks
HADM_ID 185777: 3 chunks
HADM_ID 178980: 1 chunks
HADM_ID 107064: 3 chunks
HADM_ID 150750: 5 chunks
HADM_ID 194540: 3 chunks
HADM_ID 112213: 4 chunks
HADM_ID 143045: 3 chunks
HADM_ID 194023: 1 chunks
HADM_ID 161087: 2 chunks
HADM_ID 188822: 3 chunks
HADM_ID 109235: 3 chunks
HADM_ID 109451: 5 chunks
HADM_ID 111970: 6 chunks
HADM_ID 165315: 2 chunks
HADM_ID 152223: 2 chunks
HADM_ID 124321: 4 chunks
HADM_ID 161859: 1 chunks
HADM_ID 129635: 2 chunks
HADM_ID 134931: 1 chunks
HADM_ID 162569: 2 chunks
HADM_ID 104557: 3 chunks
HADM_ID 128652: 4 chunks
HADM_ID 175413: 4 chunks
HADM_ID 176176: 2 chunks
HADM_ID 166707: 4 chunks
HADM_ID 182104: 3 chunks
HADM_ID 122659: 7 chunks
HADM_ID 165660: 3 chunks
HADM_ID 188670: 3 chunks
HADM_ID 106266: 5 chunks
HADM_ID 119203: 2 chunks
HADM_ID 181750: 2 chunks
HADM_ID 157907: 2 chunks
HADM_ID 144073: 2 chunks
HADM_ID 190539: 2 chunks
HADM_ID 132761: 1 chunks
HADM_ID 190797: 3 chunks
HADM_ID 155385: 2 chunks
HADM_ID 190665: 2 chunks


## Evaluation

### Retrieval Module Effectiveness

In [22]:
test_preprocessed = pd.DataFrame(preprocess_rag.transform(test), columns=preprocess_rag.get_feature_names_out())

In [22]:
def calculate_retrieval_metrics(row, k):
    page_content, metadata = create_entry(row)
    results_k = vector_store.similarity_search_with_score(page_content, k=k)

    precision_k_opioid, precision_k_nonopioid, meansim_k = 0, 0, 0
    for doc, score in results_k:
        precision_k_opioid += (metadata["is_opioid"] == doc.metadata["is_opioid"])
        precision_k_nonopioid += (metadata["is_nonopioid"] == doc.metadata["is_nonopioid"])
        meansim_k += 1 - score #Since similarity_search_with_score returns cosine distance, not cosine similarity

    precision_k_opioid /= k
    precision_k_nonopioid /= k
    meansim_k /= k

    return precision_k_opioid, precision_k_nonopioid, meansim_k

In [24]:
k_values = [3, 5, 10]
overall_precision_k_opioid = {k: 0 for k in k_values}
overall_precision_k_nonopioid = {k: 0 for k in k_values}
overall_meansim_k = {k: 0 for k in k_values}

for _, row in test_preprocessed.iterrows():
    print(f"HADM_ID {row['HADM_ID']}")
    for k in k_values:
        precision_k_opioid, precision_k_nonopioid, meansim_k = calculate_retrieval_metrics(row, k)
        overall_precision_k_opioid[k] += precision_k_opioid
        overall_precision_k_nonopioid[k] += precision_k_nonopioid
        overall_meansim_k[k] += meansim_k

overall_precision_k_opioid    = {k: overall_precision_k_opioid[k]/len(test_preprocessed) for k in k_values}
overall_precision_k_nonopioid = {k: overall_precision_k_nonopioid[k]/len(test_preprocessed) for k in k_values}
overall_meansim_k             = {k: overall_meansim_k[k]/len(test_preprocessed) for k in k_values}

print(f"Precision@k (Opioid)    : {overall_precision_k_opioid}")
print(f"Precision@k (Non-opioid): {overall_precision_k_nonopioid}")
print(f"MeanSim@k               : {overall_meansim_k}")

HADM_ID 163353
HADM_ID 118037
HADM_ID 159514
HADM_ID 103251
HADM_ID 144319
HADM_ID 115799
HADM_ID 146828
HADM_ID 198892
HADM_ID 172056
HADM_ID 143430
HADM_ID 104518
HADM_ID 190201
HADM_ID 111944
HADM_ID 175016
HADM_ID 190659
HADM_ID 142807
HADM_ID 160481
HADM_ID 183686
HADM_ID 140037
HADM_ID 127870
HADM_ID 187373
HADM_ID 145167
HADM_ID 138376
HADM_ID 172461
HADM_ID 112906
HADM_ID 134369
HADM_ID 141647
HADM_ID 171781
HADM_ID 103631
HADM_ID 191611
HADM_ID 128219
HADM_ID 148505
HADM_ID 110104
HADM_ID 118936
HADM_ID 186970
HADM_ID 146393
HADM_ID 111255
HADM_ID 101828
HADM_ID 186539
HADM_ID 199680
HADM_ID 151459
HADM_ID 182562
HADM_ID 191151
HADM_ID 112013
HADM_ID 164716
HADM_ID 171601
HADM_ID 162308
HADM_ID 199765
HADM_ID 173307
HADM_ID 193264
HADM_ID 146586
HADM_ID 121438
HADM_ID 184134
HADM_ID 197569
HADM_ID 166434
HADM_ID 112625
HADM_ID 183291
HADM_ID 107660
HADM_ID 176203
HADM_ID 169339
HADM_ID 119310
HADM_ID 108923
HADM_ID 112976
HADM_ID 131488
HADM_ID 159476
HADM_ID 159730
HADM_ID 11

### Predictive Performance on Binary Classification Tasks

#### Baseline

In [54]:
model = Pipeline([
    ("preprocess", preprocess_baseline),
    ("classif", GradientBoostingClassifier(random_state=351))
])
model.fit(train, train["is_opioid"])

,steps,"[('preprocess', ...), ('classif', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('cat', ...), ('num', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,False


In [55]:
print(classification_report(test["is_opioid"], model.predict(test), digits=3))
print(f"AUROC: {roc_auc_score(test["is_opioid"], model.predict_proba(test)[:, 1]):.3f}")

              precision    recall  f1-score   support

       False      0.947     0.421     0.583       928
        True      0.659     0.979     0.788      1058

    accuracy                          0.719      1986
   macro avg      0.803     0.700     0.685      1986
weighted avg      0.793     0.719     0.692      1986

AUROC: 0.738


In [56]:
model = Pipeline([
    ("preprocess", preprocess_baseline),
    ("classif", GradientBoostingClassifier(random_state=351))
])
model.fit(train, train["is_nonopioid"])

,steps,"[('preprocess', ...), ('classif', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('cat', ...), ('num', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,False


In [58]:
print(classification_report(test["is_nonopioid"], model.predict(test), digits=3))
print(f"AUROC: {roc_auc_score(test["is_nonopioid"], model.predict_proba(test)[:, 1]):.3f}")

              precision    recall  f1-score   support

       False      0.932     0.485     0.638       791
        True      0.741     0.977     0.843      1195

    accuracy                          0.781      1986
   macro avg      0.837     0.731     0.741      1986
weighted avg      0.817     0.781     0.761      1986

AUROC: 0.768


#### RAG-LLM

In [3]:
IP_ADDRESS = "localhost"
PORT = 11434
ENDPOINT = "/api/chat"
URL = f"http://{IP_ADDRESS}:{PORT}{ENDPOINT}"
HEADERS = {"Content-Type": "application/json"}

MODEL = "llama3.1:8b"

In [4]:
def format_query(patient, context):
    combined_context = "\n\n".join([
        f"Case:{case.page_content}\nTreatment:{{is_opioid: {case.metadata['is_opioid']}, is_nonopioid: {case.metadata['is_nonopioid']}}}"
        for case in context
    ])
    
    return f"<Patient>{patient}</Patient><Context>{combined_context}</Context>"

In [ ]:
def generate_binary_inference(url, headers, model, content, retry_count=0, retry_limit=3, timeout=1000):
    try:
        response = requests.post(
        timeout=timeout,
        url=url,
        headers=headers,
        data=json.dumps({
            "model": model,
            "options": {"seed": 351, "temperature": 0},
            "stream": False,
            "messages": [
                {
                    "role": "system",
                    "content": "You are an expert at prescribing medicine. You suggest a treatment plan for the given <Patient> based on similar historical cases <Context>. Indicate 'is_opioid: true' if the <Patient> needs an opioid painkiller and 'is_opioid: false' otherwise. Indicate 'is_nonopioid: true' if the <Patient> needs a non-opioid painkiller and 'is_nonopioid: false' otherwise.",
                },
                {
                    "role": "user",
                    "content": content
                }
            ],
            "format": {
                "type": "object",
                "properties": {
                    "is_opioid": {"type": "boolean"},
                    "is_nonopioid": {"type": "boolean"}
                },
                "required": ["is_opioid", "is_nonopioid"]
            }
        }))
        predictions_json_string = response.json()["message"]["content"]
        return json.loads(predictions_json_string)

    except requests.Timeout:
        print(f"{retry_count} | Timed out")
        if retry_count <= retry_limit: return generate_binary_inference(url, headers, model, content, retry_count + 1)

    except requests.JSONDecodeError:
        print(response.reason)
        print(response.text)

In [5]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})

##### Run 1 (num_ctx=4096)

In [46]:
all_predictions = {"is_opioid": [], "is_nonopioid": []}

for _, row in test_preprocessed.iterrows():
    patient, metadata = create_entry(row)
    context = retriever.invoke(patient)
    predictions = generate_binary_inference(url=URL, headers=HEADERS, model=MODEL, content=format_query(patient, context))

    all_predictions["is_opioid"].append(predictions["is_opioid"])
    all_predictions["is_nonopioid"].append(predictions["is_nonopioid"])
    print(f"HADM_ID {row['HADM_ID']}: Predicted: {predictions['is_opioid'], predictions['is_nonopioid']}, Actual: {metadata['is_opioid'], metadata['is_nonopioid']}")

HADM_ID 163353: Predicted: (False, True), Actual: (False, False)
HADM_ID 118037: Predicted: (False, False), Actual: (False, False)
HADM_ID 159514: Predicted: (False, True), Actual: (False, False)
HADM_ID 103251: Predicted: (False, True), Actual: (False, False)
HADM_ID 144319: Predicted: (True, False), Actual: (True, True)
HADM_ID 115799: Predicted: (True, True), Actual: (False, True)
HADM_ID 146828: Predicted: (True, True), Actual: (True, True)
HADM_ID 198892: Predicted: (False, True), Actual: (False, False)
HADM_ID 172056: Predicted: (True, True), Actual: (True, True)
HADM_ID 143430: Predicted: (True, True), Actual: (True, True)
HADM_ID 104518: Predicted: (False, False), Actual: (False, False)
HADM_ID 190201: Predicted: (False, True), Actual: (False, False)
HADM_ID 111944: Predicted: (True, True), Actual: (False, False)
HADM_ID 175016: Predicted: (True, True), Actual: (True, True)
HADM_ID 190659: Predicted: (False, True), Actual: (False, False)
HADM_ID 142807: Predicted: (False, True)

In [47]:
pd.DataFrame(all_predictions).to_csv("llm-rag_predictions.csv", index=False)

In [51]:
print(classification_report(test["is_opioid"], all_predictions["is_opioid"], digits=3))
print(f"AUROC: {roc_auc_score(test["is_opioid"], all_predictions["is_opioid"]):.3f}")

              precision    recall  f1-score   support

       False      0.656     0.610     0.632       928
        True      0.678     0.719     0.698      1058

    accuracy                          0.668      1986
   macro avg      0.667     0.665     0.665      1986
weighted avg      0.667     0.668     0.667      1986

AUROC: 0.665


In [52]:
print(classification_report(test["is_nonopioid"], all_predictions["is_nonopioid"], digits=3))
print(f"AUROC: {roc_auc_score(test["is_nonopioid"], all_predictions["is_nonopioid"]):.3f}")

              precision    recall  f1-score   support

       False      0.475     0.190     0.271       791
        True      0.616     0.861     0.718      1195

    accuracy                          0.594      1986
   macro avg      0.545     0.525     0.495      1986
weighted avg      0.560     0.594     0.540      1986

AUROC: 0.525


##### Run 2 (num_ctx=16384)

In [24]:
queries = {"is_opioid": [], "is_nonopioid": [], "query": []}

for _, row in test_preprocessed.iterrows():
    patient, metadata = create_entry(row)
    context = retriever.invoke(patient)
    
    queries["is_opioid"].append(metadata["is_opioid"])
    queries["is_nonopioid"].append(metadata["is_nonopioid"])
    queries["query"].append(format_query(patient, context))
    print(f"HADM_ID {row['HADM_ID']}")

HADM_ID 163353
HADM_ID 118037
HADM_ID 159514
HADM_ID 103251
HADM_ID 144319
HADM_ID 115799
HADM_ID 146828
HADM_ID 198892
HADM_ID 172056
HADM_ID 143430
HADM_ID 104518
HADM_ID 190201
HADM_ID 111944
HADM_ID 175016
HADM_ID 190659
HADM_ID 142807
HADM_ID 160481
HADM_ID 183686
HADM_ID 140037
HADM_ID 127870
HADM_ID 187373
HADM_ID 145167
HADM_ID 138376
HADM_ID 172461
HADM_ID 112906
HADM_ID 134369
HADM_ID 141647
HADM_ID 171781
HADM_ID 103631
HADM_ID 191611
HADM_ID 128219
HADM_ID 148505
HADM_ID 110104
HADM_ID 118936
HADM_ID 186970
HADM_ID 146393
HADM_ID 111255
HADM_ID 101828
HADM_ID 186539
HADM_ID 199680
HADM_ID 151459
HADM_ID 182562
HADM_ID 191151
HADM_ID 112013
HADM_ID 164716
HADM_ID 171601
HADM_ID 162308
HADM_ID 199765
HADM_ID 173307
HADM_ID 193264
HADM_ID 146586
HADM_ID 121438
HADM_ID 184134
HADM_ID 197569
HADM_ID 166434
HADM_ID 112625
HADM_ID 183291
HADM_ID 107660
HADM_ID 176203
HADM_ID 169339
HADM_ID 119310
HADM_ID 108923
HADM_ID 112976
HADM_ID 131488
HADM_ID 159476
HADM_ID 159730
HADM_ID 11

In [25]:
pd.DataFrame(queries).to_csv("queries.csv", index=False)

*Actual performing of Run 2 was done in Google Colab using Nunez_Project_Run_2.ipynb due to inference speed in local computer.

In [30]:
preds_16384 = pd.read_csv("llm-rag_predictions_16384.csv")

In [31]:
print(classification_report(test["is_opioid"], preds_16384["is_opioid"], digits=3))
print(f"AUROC: {roc_auc_score(test["is_opioid"], preds_16384["is_opioid"]):.3f}")

              precision    recall  f1-score   support

       False      0.683     0.597     0.637       928
        True      0.682     0.757     0.717      1058

    accuracy                          0.682      1986
   macro avg      0.682     0.677     0.677      1986
weighted avg      0.682     0.682     0.680      1986

AUROC: 0.677


In [32]:
print(classification_report(test["is_nonopioid"], preds_16384["is_nonopioid"], digits=3))
print(f"AUROC: {roc_auc_score(test["is_nonopioid"], preds_16384["is_nonopioid"]):.3f}")

              precision    recall  f1-score   support

       False      0.390     0.377     0.383       791
        True      0.597     0.610     0.603      1195

    accuracy                          0.517      1986
   macro avg      0.493     0.493     0.493      1986
weighted avg      0.514     0.517     0.516      1986

AUROC: 0.493
